In [2]:
from pyspark.sql import SparkSession


database = "hacking"
collection = "sales"
connectionString = "mongodb://127.0.0.1"

spark = SparkSession \
    .builder \
    .appName("aggregate_dataframe") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
    .config("spark.mongodb.input.uri", connectionString) \
    .config("spark.mongodb.output.uri", connectionString) \
    .getOrCreate()


:: loading settings :: url = jar:file:/Users/tmillar/dev/repo/hacking-spark-with-mongo/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tmillar/.ivy2/cache
The jars for the packages stored in: /Users/tmillar/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8c7550e-7f96-45e1-b63c-ca16c32a3b46;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 1777ms :: artifacts dl 6ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.5 from central in [default]
	---------------------------------------------------------------------
	|                 

22/11/11 20:29:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/11 20:29:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.format("mongodb") \
    .option("database", database) \
    .option("collection", collection) \
    .load()

In [4]:
df.show()
df.printSchema()

+-------+------+-----+--------------------+
|Company|Person|Sales|                 _id|
+-------+------+-----+--------------------+
|   GOOG|   Joe|  200|636d401e920e9fad5...|
|   GOOG|  Jane|  120|636d401e920e9fad5...|
|   GOOG|  Jake|  340|636d401e920e9fad5...|
|   MSFT|   Ace|  600|636d401e920e9fad5...|
|   MSFT|  Andy|  123|636d401e920e9fad5...|
|   MSFT|George|  222|636d401e920e9fad5...|
|     FB|  John|  210|636d401e920e9fad5...|
|     FB|   Sam|  999|636d401e920e9fad5...|
|   APPL|Johnny|  210|636d401e920e9fad5...|
|   APPL|  Mike|  700|636d401e920e9fad5...|
|   APPL|  Jane|  100|636d401e920e9fad5...|
|   APPL| Chris|   70|636d401e920e9fad5...|
+-------+------+-----+--------------------+

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- _id: string (nullable = true)



Group by company, will infer the numeric column for aggregating

In [5]:
df.groupBy("Company")
df.groupBy("Company").count().show()
df.groupBy("Company").avg().show()
df.groupBy("Company").max().show()
df.groupBy("Company").mean().show()
df.groupBy("Company").sum().show()


+-------+-----+
|Company|count|
+-------+-----+
|   GOOG|    3|
|   MSFT|    3|
|     FB|    2|
|   APPL|    4|
+-------+-----+

+-------+----------+
|Company|avg(Sales)|
+-------+----------+
|   GOOG|     220.0|
|   MSFT|     315.0|
|     FB|     604.5|
|   APPL|     270.0|
+-------+----------+

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|       340|
|   MSFT|       600|
|     FB|       999|
|   APPL|       700|
+-------+----------+

+-------+----------+
|Company|avg(Sales)|
+-------+----------+
|   GOOG|     220.0|
|   MSFT|     315.0|
|     FB|     604.5|
|   APPL|     270.0|
+-------+----------+

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   GOOG|       660|
|   MSFT|       945|
|     FB|      1209|
|   APPL|      1080|
+-------+----------+



Aggregate by column and function

In [6]:
df.agg({"Sales": "sum"}).show()
df.agg({"Sales": "min"}).show()
df.agg({"Sales": "max"}).show()
df.agg({"Sales": "mean"}).show()

+----------+
|sum(Sales)|
+----------+
|      3894|
+----------+

+----------+
|min(Sales)|
+----------+
|        70|
+----------+

+----------+
|max(Sales)|
+----------+
|       999|
+----------+

+----------+
|avg(Sales)|
+----------+
|     324.5|
+----------+



More specific GroupBy where spark can't infer the numeric column to aggregate

_this will have the same results as the GroupBy above because the column here is also the infered column_

In [7]:
df_grouped = df.groupBy("Company")
df_grouped.agg({"Sales": "count"}).show()
df_grouped.agg({"Sales": "avg"}).show()
df_grouped.agg({"Sales": "max"}).show()
df_grouped.agg({"Sales": "mean"}).show()
df_grouped.agg({"Sales": "sum"}).show()


+-------+------------+
|Company|count(Sales)|
+-------+------------+
|   GOOG|           3|
|   MSFT|           3|
|     FB|           2|
|   APPL|           4|
+-------+------------+

+-------+----------+
|Company|avg(Sales)|
+-------+----------+
|   GOOG|     220.0|
|   MSFT|     315.0|
|     FB|     604.5|
|   APPL|     270.0|
+-------+----------+

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|       340|
|   MSFT|       600|
|     FB|       999|
|   APPL|       700|
+-------+----------+

+-------+----------+
|Company|avg(Sales)|
+-------+----------+
|   GOOG|     220.0|
|   MSFT|     315.0|
|     FB|     604.5|
|   APPL|     270.0|
+-------+----------+

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   GOOG|       660|
|   MSFT|       945|
|     FB|      1209|
|   APPL|      1080|
+-------+----------+



Using builtin SQL functions

In [11]:
from pyspark.sql.functions import countDistinct, avg, stddev

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_38_64.pyx", line 1589, in _pydevd_bundle.pydevd_cython_darwin_38_64.ThreadTracer.__call__
  File "_pydevd_bundle/pydevd_cython_darwin_38_64.pyx", line 756, in _pydevd_bundle.pydevd_cython_darwin_38_64.PyDBFrame.trace_dispatch
  File "/Applications/Code/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 96, in can_not_skip
    cell_info = pydb.cell_info
AttributeError: 'PyDB' object has no attribute 'cell_info'
Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_38_64.pyx", line 1589, in _pydevd_bundle.pydevd_cython_darwin_38_64.ThreadTracer.__call__
  File "_pydevd_bundle/pydevd_cython_darwin_38_64.pyx", line 756, in _pydevd_bundle.pydevd_cython_darwin_38_64.PyDBFrame.trace_dispatch
  File "/Applications/Code/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 96, in can_not_skip
    cell_info = pydb

In [14]:
df.select(countDistinct("Sales")).show()
df.select(avg("Sales")).show()
df.select(avg("Sales").alias("Better Column Name")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+

+----------+
|avg(Sales)|
+----------+
|     324.5|
+----------+

+------------------+
|Better Column Name|
+------------------+
|             324.5|
+------------------+



In [21]:
from pyspark.sql.functions import format_number

sales_std = df.select(stddev("Sales").alias("StdDev"))
sales_std.show()
sales_std.select(format_number("StdDev", 2)).show()

+-----------------+
|           StdDev|
+-----------------+
|289.4997252314601|
+-----------------+

+------------------------+
|format_number(StdDev, 2)|
+------------------------+
|                  289.50|
+------------------------+



In [24]:
df.orderBy("Sales").show()
df.orderBy(df["Sales"].desc()).show()



+-------+------+-----+--------------------+
|Company|Person|Sales|                 _id|
+-------+------+-----+--------------------+
|   APPL| Chris|   70|636d401e920e9fad5...|
|   APPL|  Jane|  100|636d401e920e9fad5...|
|   GOOG|  Jane|  120|636d401e920e9fad5...|
|   MSFT|  Andy|  123|636d401e920e9fad5...|
|   GOOG|   Joe|  200|636d401e920e9fad5...|
|     FB|  John|  210|636d401e920e9fad5...|
|   APPL|Johnny|  210|636d401e920e9fad5...|
|   MSFT|George|  222|636d401e920e9fad5...|
|   GOOG|  Jake|  340|636d401e920e9fad5...|
|   MSFT|   Ace|  600|636d401e920e9fad5...|
|   APPL|  Mike|  700|636d401e920e9fad5...|
|     FB|   Sam|  999|636d401e920e9fad5...|
+-------+------+-----+--------------------+

+-------+------+-----+--------------------+
|Company|Person|Sales|                 _id|
+-------+------+-----+--------------------+
|     FB|   Sam|  999|636d401e920e9fad5...|
|   APPL|  Mike|  700|636d401e920e9fad5...|
|   MSFT|   Ace|  600|636d401e920e9fad5...|
|   GOOG|  Jake|  340|636d401e9